In [ ]:
!pip install pyttsx3
!pip install SpeechRecognition

In [ ]:
!pip install pyaudio

In [ ]:
!pip install nltk

In [ ]:
responses = []

In [ ]:
import speech_recognition as sr
import pyttsx3

#initialize speech recognition engine and text to speech engine
r=sr.Recognizer()
engine=pyttsx3.init()

#set the properties for the text to spech engine
voices=engine.getProperty('voices')
engine.setProperty('voice',voices[0].id)
engine.setProperty('rate',150)

def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Speak now:")
        audio = recognizer.listen(source)

    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        print("Sorry, I couldn't understand what you said.")
        return ""
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
        return ""

def speak(text):
    engine.say(text)
    engine.runAndWait()
speak("Hello,I'm your personnel A I.")
questions = [     
                  "How often have you felt nervous or anxious?",
                  "Have you felt hopeless about your future?",
                  "Have you experienced mood swings?",
                  "Do you feel unable to cope up with daily problems?",
                  "Do you feel mentally weak when interacting with others?",
                  
]




for question in questions:
   print(question)
   speak(question)
   response = speech_to_text()
   responses.append(response)
   print(response)

speak("Thank you for your responses. Here are your answers:")
for i in range(len(questions)):
   print(f"{questions[i]} {responses[i]}")



In [ ]:
responses

In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
stopwords.words('english')

In [ ]:
responses1 = []
stop_words = stopwords.words('english')
for words in responses:
  if words not in stop_words:
   responses1.append(words)
responses1

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,3))

In [ ]:
x1=tfidf.fit_transform(responses1)
x1

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df1=pd.DataFrame(x1[0].T.todense(),index=tfidf.get_feature_names_out(),columns=["TfIdf value"] )
df1=df1.sort_values('TfIdf value',ascending=False)
df1

In [ ]:
df1.head(20)

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
wordcloud = WordCloud().generate(str(responses1))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
def find_largest(neg, neu, pos):
    if neg >= neu and neg >= pos:
        return {'negative' : neg}
    elif neu >= neg and neu >= pos:
        return {'neutral' : neu}
    else:
        return {'positive' : pos}

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

model1 = SentimentIntensityAnalyzer()

In [ ]:
responses_string = " ".join(responses1)
vader_scores = model1.polarity_scores(responses_string)
vader_scores

In [ ]:
vader_largest = find_largest(vader_scores['neg'],vader_scores['neu'],vader_scores['pos'])
vader_largest

In [ ]:
!pip install scipy

In [ ]:
!pip install transformers

In [ ]:
!pip install torch

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
Model = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(Model)
model = AutoModelForSequenceClassification.from_pretrained(Model)

In [ ]:
text_data = tokenizer(responses_string,return_tensors='pt')
text_data

In [ ]:
result = model(**text_data)
result

In [ ]:
result[0]

In [ ]:
score = result[0][0].detach().numpy()
score = softmax(score)
score

In [ ]:
label = {
    'neg' : score[0],
    'neu' : score[1],
    'pos' : score[2]
}

print(label)

In [ ]:
roberta_largest = find_largest(label['neg'],label['neu'],label['pos'])
roberta_largest

In [ ]:
total = []
total.append(list(vader_largest)[0])
total.append(list(roberta_largest)[0])

print("Total scores : \n vader = {0}\n roberta = {1} \n {2}".format(vader_largest,roberta_largest,total))

In [ ]:
neutral1 = ['neutral','neutral']
negative1 = [['neutral','negative'],['negative','neutral'],['negative','negative']]
positive1 = [['positive','positive'],['neutral','positive'],['positive','neutral']]

In [ ]:
if total in neutral1:
  print("Neutral")

elif total in negative1:
  print("Negative")

else:
  print("Positive")